In [1]:
import pandas as pd
import numpy as np
import allel
import moments.LD

### Preprocessing of 1000G VCF files
1. Apply strict mask
    - Some sites are not reliably mapped in hg38 and needs to be excluded.
    - These regions include: repeat regions, pseudogenes, centromeric regions.
2. Select only biallelic SNP sites
    - A VCF can contain mult-allelic SNPs and structure variants (indels)
    - The biallelic SNP sites are the easiest to work with, and most studies focus on them.